In [61]:
import numpy as np
import scipy.linalg as la
import math
import dimod
from dimod import SimulatedAnnealingSampler

from qubo_helpers import *

In [62]:
Q, q, kappa_vec, x_star = generate_quip(n = 10, kappa = 100)
optimal_energy = x_star.T @ Q @ x_star + q.T @ x_star

Gradient at x* (should be ~0): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [63]:
Q.shape, q.shape, x_star, optimal_energy

((10, 10),
 (10,),
 array([ 0,  0, 37,  0, 32,  0,  0,  0,  0, 69]),
 np.float64(-42837.0))

### BINARY

In [64]:
width, coefficients = create_binary_encoder(kappa = 100)
coefficients

[1, 2, 4, 8, 16, 32, 64]

In [65]:
C = create_binary_encoding_matrix(10, width, coefficients)
C.shape

(10, 70)

In [66]:
Q_binary = C.T @ Q @ C
q_binary = C.T @ q

Q_binary.shape, q_binary.shape

((70, 70), (70,))

In [67]:
Q_qubo = convert_to_qubo(Q_binary, q_binary)
Q_qubo.shape

binaryQubo = Q_qubo

### BOUNDED

In [68]:
n = 10
kappa = 100
ell = 0.01
c = 0.01

In [69]:
print(f"Using precision parameters: ℓ={ell}, c={c}")
mu_vec = find_coefficient_upper_bounds(Q, q, kappa_vec, ell, c)

Using precision parameters: ℓ=0.01, c=0.01
ml (min linear coeff magnitude): 200.000000
mc (min quadratic coeff magnitude): 0.500000
Initial μ bounds: [2.88675135 2.67261242 2.88675135 2.88675135 3.53553391 4.0824829
 3.53553391 2.88675135 2.88675135 2.67261242]
Final μ bounds after constraint satisfaction: [2 2 2 2 3 4 3 2 2 2]


In [70]:
all_coefficients, widths, total_binary_vars = create_bounded_encoding_system(kappa_vec, mu_vec = [8, 8, 8, 8, 8, 8, 8, 8, 8, 8])


=== CREATING BOUNDED COEFFICIENT ENCODING SYSTEM ===
  Encoding variable with κ=100, μ=8
    Using bounded-coefficient encoding
    ρ=4, ν=85, η=10
    Coefficients: [1, 2, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, np.int64(5)]
Variable x[0]: κ=100, μ=8, width=15, coeffs=[1, 2, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, np.int64(5)]
  Encoding variable with κ=100, μ=8
    Using bounded-coefficient encoding
    ρ=4, ν=85, η=10
    Coefficients: [1, 2, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, np.int64(5)]
Variable x[1]: κ=100, μ=8, width=15, coeffs=[1, 2, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, np.int64(5)]
  Encoding variable with κ=100, μ=8
    Using bounded-coefficient encoding
    ρ=4, ν=85, η=10
    Coefficients: [1, 2, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, np.int64(5)]
Variable x[2]: κ=100, μ=8, width=15, coeffs=[1, 2, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, np.int64(5)]
  Encoding variable with κ=100, μ=8
    Using bounded-coefficient encoding
    ρ=4, ν=85, η=10
    Coefficients: [1, 2, 4, 8, 8, 8, 8, 8, 8

In [71]:
len([8, 8, 8, 8, 8, 8, 8, 8, 8, 8])

10

In [72]:
C = create_bounded_encoding_matrix(n, all_coefficients, widths)
C.shape

(10, 150)

In [73]:
Q_binary = C.T @ Q @ C
q_binary = C.T @ q

Q_binary.shape, q_binary.shape

((150, 150), (150,))

In [74]:
print(f"Range of Q_binary coefficients: [{np.min(Q_binary):.2f}, {np.max(Q_binary):.2f}]")
print(f"Range of q_binary coefficients: [{np.min(q_binary):.2f}, {np.max(q_binary):.2f}]")

Range of Q_binary coefficients: [-128.00, 448.00]
Range of q_binary coefficients: [-7472.00, 1064.00]


In [75]:
Q_qubo = convert_to_qubo(Q_binary, q_binary)
boundedQubo = Q_qubo

In [77]:
x_1 = np.array([0, 0, 38, 0, 33, 0, 13, 0, 0, 68])
x_2 = np.array([0, 0, 38, 0, 31, 0, 13, 0, 0, 67])

In [95]:
objective = lambda x : x.T @ Q @ x + q.T @ x
gradient = lambda x : 2 * Q @ x + q

In [87]:
results = {
    "boundedEnergies" : [],
    "binaryEnergies" : [],
    "boundedSols" : [],
    "binarySols" : []
}

In [88]:
binaryBqm = dimod.BQM.from_qubo(binaryQubo)
boundedBqm = dimod.BQM.from_qubo(boundedQubo)

In [99]:
# BINARY
for _ in range(10):
    sampler = SimulatedAnnealingSampler()
    sampleset = sampler.sample(binaryBqm)
    results["binaryEnergies"].append(sampleset.first.energy)
    results["binarySols"].append(sampleset.first)

#BOUNDED
for _ in range(10):
    sampler = SimulatedAnnealingSampler()
    sampleset = sampler.sample(boundedBqm)
    results["boundedEnergies"].append(sampleset.first.energy)
    results["boundedSols"].append(sampleset.first)

In [101]:
np.mean(results["boundedEnergies"]), np.std(results["boundedEnergies"])

(np.float64(-43524.769230769234), np.float64(7.597103153923933))

In [102]:
np.mean(results["binaryEnergies"]), np.std(results["binaryEnergies"])

(np.float64(-43455.03846153846), np.float64(95.45487486815261))